# Building AI Agents with Authority Nanos

This notebook demonstrates how to build AI agents using the Authority Kernel.

## What is an AI Agent?

An AI agent is a system that:
1. Receives a task or goal
2. Uses an LLM to reason about the task
3. Executes tools to interact with the environment
4. Iterates until the task is complete

## Why Authority Nanos for Agents?

AI agents can be dangerous without proper controls. Authority Nanos provides:

| Concern | Authority Nanos Solution |
|---------|-------------------------|
| Unauthorized file access | Policy-controlled file I/O |
| Unsafe network requests | Policy-controlled HTTP |
| Runaway execution | Budget limits and timeouts |
| Lack of visibility | Comprehensive audit logging |
| Memory corruption | Typed heap with versioning |

Let's build a simple agent!

In [ ]:
# Install the SDK (uncomment if needed)
# %pip install authority-nanos

import json
from typing import Dict, List, Optional, Any
from authority_nanos import (
    AuthorityKernel,
    OperationDeniedError,
    AuthorityKernelError
)

print("Authority Nanos SDK imported successfully!")

## 1. Agent Architecture Overview

A typical agent has three main components:

```
+-------------------+
|    Agent Loop     |
|  (orchestrator)   |
+-------------------+
         |
         v
+-------------------+     +-------------------+
|   LLM Inference   |<--->|  Tool Execution   |
|   (thinking)      |     |  (actions)        |
+-------------------+     +-------------------+
         |
         v
+-------------------+
| Authority Kernel  |
| (security layer)  |
+-------------------+
```

Let's implement each component.

## 2. Tool Execution

Tools are functions the agent can call to interact with the world. In Authority Nanos, tools run in a sandboxed environment.

In [ ]:
with AuthorityKernel(simulate=True) as ak:
    print("=== Tool Execution Demo ===")
    print()
    
    # Execute the built-in 'add' tool
    result = ak.tool_call("add", {"a": 10, "b": 32})
    print(f"add(10, 32) = {json.loads(result.decode())}")
    
    # Execute the 'concat' tool
    result = ak.tool_call("concat", {"str1": "Hello, ", "str2": "Agent!"})
    print(f"concat('Hello, ', 'Agent!') = {json.loads(result.decode())}")
    
    # Execute a custom tool (returns simulated result)
    result = ak.tool_call("weather", {"city": "San Francisco"})
    print(f"weather('San Francisco') = {json.loads(result.decode())}")

### Defining Tools for an Agent

Let's define a set of tools an agent can use:

In [ ]:
# Tool definitions (for LLM to understand)
TOOLS = [
    {
        "name": "calculate",
        "description": "Perform basic arithmetic calculations",
        "parameters": {
            "expression": "A math expression like '2 + 2' or '10 * 5'"
        }
    },
    {
        "name": "read_file",
        "description": "Read contents of a file",
        "parameters": {
            "path": "The file path to read"
        }
    },
    {
        "name": "write_file",
        "description": "Write content to a file",
        "parameters": {
            "path": "The file path to write",
            "content": "The content to write"
        }
    },
    {
        "name": "search_web",
        "description": "Search the web for information",
        "parameters": {
            "query": "The search query"
        }
    }
]

print("Available tools:")
for tool in TOOLS:
    print(f"  - {tool['name']}: {tool['description']}")

### Tool Executor with Authorization

A secure tool executor checks authorization before execution:

In [ ]:
class ToolExecutor:
    """Executes tools with authorization checks."""
    
    def __init__(self, kernel: AuthorityKernel):
        self.ak = kernel
    
    def execute(self, tool_name: str, params: Dict[str, Any]) -> Dict[str, Any]:
        """Execute a tool with authorization."""
        try:
            if tool_name == "calculate":
                return self._calculate(params["expression"])
            elif tool_name == "read_file":
                return self._read_file(params["path"])
            elif tool_name == "write_file":
                return self._write_file(params["path"], params["content"])
            elif tool_name == "search_web":
                return self._search_web(params["query"])
            else:
                # Use kernel's tool_call for unknown tools
                result = self.ak.tool_call(tool_name, params)
                return json.loads(result.decode())
        except OperationDeniedError as e:
            return {"error": f"Permission denied: {e}", "denied": True}
        except Exception as e:
            return {"error": str(e)}
    
    def _calculate(self, expression: str) -> Dict[str, Any]:
        """Simple calculator (safe subset of eval)."""
        # Only allow numbers and basic operators
        allowed = set("0123456789+-*/. ()")
        if not all(c in allowed for c in expression):
            return {"error": "Invalid expression"}
        try:
            result = eval(expression)
            return {"result": result}
        except Exception as e:
            return {"error": str(e)}
    
    def _read_file(self, path: str) -> Dict[str, Any]:
        """Read file with authorization."""
        if not self.ak.authorize("read", path):
            denial = self.ak.get_last_denial()
            return {
                "error": f"Not authorized to read {path}",
                "reason": denial.reason if denial else "Unknown",
                "denied": True
            }
        
        try:
            data = self.ak.file_read(path)
            return {"content": data.decode(), "path": path}
        except Exception as e:
            return {"error": str(e)}
    
    def _write_file(self, path: str, content: str) -> Dict[str, Any]:
        """Write file with authorization."""
        if not self.ak.authorize("write", path):
            denial = self.ak.get_last_denial()
            return {
                "error": f"Not authorized to write {path}",
                "reason": denial.reason if denial else "Unknown",
                "denied": True
            }
        
        try:
            self.ak.file_write(path, content.encode())
            return {"success": True, "path": path}
        except Exception as e:
            return {"error": str(e)}
    
    def _search_web(self, query: str) -> Dict[str, Any]:
        """Simulated web search."""
        # In simulation, return mock results
        return {
            "query": query,
            "results": [
                {"title": f"Result 1 for '{query}'", "url": "https://example.com/1"},
                {"title": f"Result 2 for '{query}'", "url": "https://example.com/2"},
            ],
            "simulated": True
        }


# Test the tool executor
with AuthorityKernel(simulate=True) as ak:
    executor = ToolExecutor(ak)
    
    print("=== Tool Executor Test ===")
    print()
    
    # Test calculation
    result = executor.execute("calculate", {"expression": "2 + 2 * 10"})
    print(f"calculate('2 + 2 * 10'): {result}")
    
    # Test file read (simulated file exists)
    result = executor.execute("read_file", {"path": "/etc/config.json"})
    print(f"read_file('/etc/config.json'): {result}")
    
    # Test web search
    result = executor.execute("search_web", {"query": "Authority Nanos"})
    print(f"search_web('Authority Nanos'): {result}")

## 3. LLM Inference

The LLM is the "brain" of the agent. It decides what to do next based on the current context.

In [ ]:
with AuthorityKernel(simulate=True) as ak:
    print("=== LLM Inference Demo ===")
    print()
    
    # Create an inference request
    request = json.dumps({
        "model": "gpt-4",
        "messages": [
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": "What is 2 + 2?"}
        ],
        "max_tokens": 100
    }).encode()
    
    # Send to kernel
    response = ak.inference(request)
    result = json.loads(response.decode())
    
    print("Inference Request:")
    print(f"  Model: gpt-4")
    print(f"  Prompt: 'What is 2 + 2?'")
    print()
    print("Response:")
    print(json.dumps(result, indent=2))
    
    if result.get("simulated"):
        print()
        print("Note: This is a simulated response. Real inference requires LLM configuration.")

## 4. Building the Agent Loop

Now let's put it all together into a simple agent:

In [ ]:
class SimpleAgent:
    """A simple AI agent built on Authority Nanos."""
    
    def __init__(self, kernel: AuthorityKernel):
        self.ak = kernel
        self.executor = ToolExecutor(kernel)
        self.conversation: List[Dict] = []
        self.max_iterations = 5
    
    def run(self, task: str) -> str:
        """Run the agent on a task."""
        print(f"Agent starting task: {task}")
        print("=" * 50)
        
        # Initialize conversation
        self.conversation = [
            {
                "role": "system",
                "content": self._get_system_prompt()
            },
            {
                "role": "user",
                "content": task
            }
        ]
        
        # Agent loop
        for i in range(self.max_iterations):
            print(f"\n--- Iteration {i + 1} ---")
            
            # Get LLM response
            response = self._think()
            print(f"Agent thinks: {response[:100]}..." if len(response) > 100 else f"Agent thinks: {response}")
            
            # Parse response for tool calls
            tool_call = self._parse_tool_call(response)
            
            if tool_call:
                # Execute tool
                tool_name = tool_call["name"]
                tool_params = tool_call["params"]
                
                print(f"Executing tool: {tool_name}({tool_params})")
                result = self.executor.execute(tool_name, tool_params)
                print(f"Tool result: {result}")
                
                # Add tool result to conversation
                self.conversation.append({
                    "role": "assistant",
                    "content": f"Tool call: {tool_name}({tool_params})"
                })
                self.conversation.append({
                    "role": "user",
                    "content": f"Tool result: {json.dumps(result)}"
                })
                
                # Check if tool was denied
                if result.get("denied"):
                    print(f"Tool denied - agent will need to adapt")
            else:
                # No tool call - agent is done
                print("Agent completed task.")
                return response
        
        return "Max iterations reached"
    
    def _get_system_prompt(self) -> str:
        """Get the system prompt with tool descriptions."""
        tools_desc = "\n".join(
            f"- {t['name']}: {t['description']}" 
            for t in TOOLS
        )
        return f"""You are a helpful AI assistant. You can use these tools:

{tools_desc}

To use a tool, respond with: TOOL: tool_name(param1=value1, param2=value2)
When you have completed the task, just respond normally without a TOOL: prefix.
"""
    
    def _think(self) -> str:
        """Get LLM response for current conversation."""
        request = json.dumps({
            "model": "gpt-4",
            "messages": self.conversation,
            "max_tokens": 500
        }).encode()
        
        response = self.ak.inference(request)
        result = json.loads(response.decode())
        
        # Extract assistant message
        if "choices" in result:
            return result["choices"][0]["message"]["content"]
        return "I cannot process this request."
    
    def _parse_tool_call(self, response: str) -> Optional[Dict]:
        """Parse a tool call from the response."""
        # Simple parser for "TOOL: name(params)"
        if "TOOL:" not in response:
            return None
        
        try:
            # Extract tool call part
            tool_part = response.split("TOOL:")[1].strip()
            
            # Parse name and params
            if "(" in tool_part and ")" in tool_part:
                name = tool_part.split("(")[0].strip()
                params_str = tool_part.split("(")[1].split(")")[0]
                
                # Simple param parsing
                params = {}
                if params_str:
                    for part in params_str.split(","):
                        if "=" in part:
                            k, v = part.split("=", 1)
                            params[k.strip()] = v.strip().strip('"').strip("'")
                
                return {"name": name, "params": params}
        except Exception:
            pass
        
        return None


print("SimpleAgent class defined!")

### Running the Agent

Let's run the agent on a simple task:

In [ ]:
with AuthorityKernel(simulate=True) as ak:
    agent = SimpleAgent(ak)
    
    # Run a simple task
    # Note: In simulation mode, the LLM returns simulated responses
    result = agent.run("Calculate 15 * 7 + 3")
    
    print("\n" + "=" * 50)
    print(f"Final result: {result}")

## 5. Agent State Management

Use the typed heap to persist agent state:

In [ ]:
class StatefulAgent:
    """An agent that persists state in the typed heap."""
    
    def __init__(self, kernel: AuthorityKernel, agent_id: str):
        self.ak = kernel
        self.agent_id = agent_id
        self.state_handle = None
        self._initialize_state()
    
    def _initialize_state(self):
        """Initialize agent state in typed heap."""
        initial_state = json.dumps({
            "agent_id": self.agent_id,
            "created_at": "2024-01-01T00:00:00Z",
            "task_count": 0,
            "last_task": None,
            "memory": []
        }).encode()
        
        self.state_handle = self.ak.alloc("agent_state", initial_state)
        print(f"Agent '{self.agent_id}' initialized with handle: {self.state_handle}")
    
    def get_state(self) -> Dict:
        """Get current agent state."""
        data = self.ak.read(self.state_handle)
        return json.loads(data.decode())
    
    def record_task(self, task: str, result: str):
        """Record a completed task."""
        state = self.get_state()
        
        # Update state
        patch = json.dumps([
            {"op": "replace", "path": "/task_count", "value": state["task_count"] + 1},
            {"op": "replace", "path": "/last_task", "value": task},
            {"op": "add", "path": "/memory/-", "value": {
                "task": task,
                "result": result[:100]  # Truncate for brevity
            }}
        ]).encode()
        
        self.ak.write(self.state_handle, patch)
        print(f"Recorded task #{state['task_count'] + 1}")


# Demo stateful agent
with AuthorityKernel(simulate=True) as ak:
    print("=== Stateful Agent Demo ===")
    print()
    
    agent = StatefulAgent(ak, "agent-001")
    
    # Simulate completing tasks
    agent.record_task("Calculate 2+2", "4")
    agent.record_task("Search for weather", "Sunny, 72F")
    agent.record_task("Write report", "Report written successfully")
    
    print()
    print("Final agent state:")
    print(json.dumps(agent.get_state(), indent=2))

## 6. Agent with Authorization Constraints

Let's see how the agent handles authorization denials:

In [ ]:
with AuthorityKernel(simulate=True) as ak:
    print("=== Agent with Authorization Constraints ===")
    print()
    
    # Set up some restrictions
    ak.deny_target("/etc/passwd")
    ak.deny_target("/etc/shadow")
    ak.deny_operation("write")
    
    print("Policy configured:")
    print("  - Denied: /etc/passwd, /etc/shadow")
    print("  - Denied: all write operations")
    print()
    
    executor = ToolExecutor(ak)
    
    # Test various operations
    operations = [
        ("read_file", {"path": "/etc/config.json"}),  # Allowed
        ("read_file", {"path": "/etc/passwd"}),       # Denied
        ("write_file", {"path": "/tmp/test.txt", "content": "test"}),  # Denied
        ("calculate", {"expression": "10 + 20"}),  # Allowed
    ]
    
    for tool_name, params in operations:
        print(f"Tool: {tool_name}({params})")
        result = executor.execute(tool_name, params)
        
        if result.get("denied"):
            print(f"  DENIED: {result.get('error')}")
        elif result.get("error"):
            print(f"  ERROR: {result.get('error')}")
        else:
            print(f"  SUCCESS: {result}")
        print()

## 7. Agent Audit Trail

All agent actions are logged for auditing:

In [ ]:
with AuthorityKernel(simulate=True) as ak:
    print("=== Agent Audit Trail ===")
    print()
    
    # Run some operations
    ak.deny_target("/secrets/key.txt")
    
    executor = ToolExecutor(ak)
    executor.execute("calculate", {"expression": "2 + 2"})
    executor.execute("read_file", {"path": "/etc/config.json"})
    executor.execute("read_file", {"path": "/secrets/key.txt"})
    
    # Get audit logs
    logs = ak.audit_logs()
    
    print(f"Total audit entries: {len(logs)}")
    print()
    print("Authorization events:")
    
    for log_bytes in logs:
        entry = json.loads(log_bytes.decode())
        event = entry.get("event", "")
        
        if "authorize" in event:
            details = entry.get("details", {})
            status = "ALLOWED" if details.get("authorized") else "DENIED"
            op = details.get("operation", "?")
            target = details.get("target", "?")
            print(f"  [{entry['timestamp'][:19]}] {op} {target}: {status}")

## Summary

In this notebook, you learned how to:

1. **Execute tools** through the Authority Kernel
2. **Define and implement tools** with authorization checks
3. **Use LLM inference** for agent reasoning
4. **Build an agent loop** that coordinates thinking and actions
5. **Persist agent state** in the typed heap
6. **Handle authorization denials** gracefully
7. **Audit agent actions** for visibility and compliance

## Key Patterns

- Always check authorization before performing sensitive operations
- Use the typed heap for agent state persistence
- Handle tool execution failures gracefully
- Log important events for debugging and auditing

## Next Steps

- **04_langchain_integration.ipynb** - Integrate with LangChain for production agents